In [25]:
import pandas as pd
import numpy as np
# 读取数据
train_df = pd.read_csv('train_dic.csv')
test_df = pd.read_csv('test_dic.csv')

In [2]:
(train_df['sales'] < 0).sum()

0

In [3]:
train_df.columns

Index(['unique_id', 'date', 'warehouse', 'total_orders', 'sales',
       'sell_price_main', 'holiday_name', 'holiday', 'shops_closed',
       'winter_school_holidays', 'school_holidays', 'product_unique_id',
       'name', 'L1_category_name_en', 'L2_category_name_en',
       'L3_category_name_en', 'L4_category_name_en', 'year', 'month_sin',
       'month_cos', 'day_sin', 'day_cos', 'is_discount', 'max_discount'],
      dtype='object')

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4007367 entries, 0 to 4007366
Data columns (total 24 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   unique_id               int64  
 1   date                    object 
 2   warehouse               object 
 3   total_orders            float64
 4   sales                   float64
 5   sell_price_main         float64
 6   holiday_name            object 
 7   holiday                 bool   
 8   shops_closed            bool   
 9   winter_school_holidays  bool   
 10  school_holidays         bool   
 11  product_unique_id       int64  
 12  name                    object 
 13  L1_category_name_en     object 
 14  L2_category_name_en     object 
 15  L3_category_name_en     object 
 16  L4_category_name_en     object 
 17  year                    object 
 18  month_sin               float64
 19  month_cos               float64
 20  day_sin                 float64
 21  day_cos                 float64

In [5]:
train_df['warehouse'].value_counts()

warehouse
Prague_1       780566
Prague_3       779655
Prague_2       770709
Brno_1         643637
Budapest_1     574582
Munich_1       259287
Frankfurt_1    198931
Name: count, dtype: int64

In [6]:
import numpy as np

# 将日期转换为 datetime 格式
train_df['date'] = pd.to_datetime(train_df['date'])
test_df['date'] = pd.to_datetime(test_df['date'])

# 提取日期中的“日”
train_df['day'] = train_df['date'].dt.day
test_df['day'] = test_df['date'].dt.day

# 计算全局均值
global_mean = train_df['sales'].mean()

# 对每个列进行编码
for col in ['warehouse', 'name']:

    # 计算每个 (col, day) 组合的均值
    col_day_mean = train_df.groupby([col, 'day'])['sales'].mean().reset_index()
    col_day_mean = col_day_mean.rename(columns={'sales': col + '_day_mean'})

    # # 在计算均值后加入 log 平滑
    # col_day_mean[col + '_day_mean'] = np.log1p(col_day_mean[col + '_day_mean'])

    # 将均值映射到训练集
    train_df = train_df.merge(col_day_mean, on=[col, 'day'], how='left')

    # 将均值映射到测试集
    test_df = test_df.merge(col_day_mean, on=[col, 'day'], how='left')

In [7]:
train_df.shape, test_df.shape

((4007367, 27), (47021, 26))

In [8]:
test_df['name_day_mean'].value_counts(), test_df['name_day_mean'].min(), test_df['name_day_mean'].max()

(name_day_mean
 20.113750     8
 25.653333     8
 83.512500     6
 69.527500     5
 28.203750     5
              ..
 211.360000    1
 212.068125    1
 226.070625    1
 191.615625    1
 257.215000    1
 Name: count, Length: 21834, dtype: int64,
 2.282482269503546,
 12452.73)

In [9]:
# 计算每个产品名称和年份组合的销售均值
product_year_mean = train_df.groupby(['name', 'year'])['sales'].mean().reset_index()

# 重命名均值列为 'product_year_encoded'
product_year_mean.rename(columns={'sales': 'year_encoded'}, inplace=True)

# # 对均值进行log平滑处理
# product_year_mean['year_encoded'] = np.log1p(product_year_mean['year_encoded'])

# 将均值合并到训练集和测试集
train_df = train_df.merge(product_year_mean, on=['name', 'year'], how='left')
test_df = test_df.merge(product_year_mean, on=['name', 'year'], how='left')

# 显示计算的均值
product_year_mean


,name,year,year_encoded
0,Apple_0,year_2021,69.192825
1,Apple_0,year_2022,97.764780
2,Apple_0,year_2023,120.743359
3,Apple_0,year_2024,166.748227
4,Apple_1,year_2022,27.061613
...,...,...,...
9065,Zucchini_8,year_2022,245.472632
9066,Zucchini_8,year_2023,280.570317
9067,Zucchini_8,year_2024,211.787561
9068,Zucchini_9,year_2022,82.325764


In [10]:
product_year_mean['year_encoded'].value_counts(), product_year_mean['year_encoded'].min(), product_year_mean['year_encoded'].max()

(year_encoded
 0.000000      15
 37.320000      2
 44.027610      1
 26.604869      1
 17.394185      1
               ..
 43.988454      1
 50.262158      1
 121.184535     1
 79.279387      1
 134.958723     1
 Name: count, Length: 9055, dtype: int64,
 0.0,
 15642.277792207791)

In [11]:
# 计算每个节假日和产品名称组合的销售均值
holiday_name_product_mean = train_df.groupby(['holiday_name', 'name'])['sales'].mean().reset_index()

# 重命名均值列为 'holiday_name_encoded'
holiday_name_product_mean.rename(columns={'sales': 'holiday_name_encoded'}, inplace=True)
# # 对均值进行log平滑处理
# holiday_name_product_mean['holiday_name_encoded'] = np.log1p(holiday_name_product_mean['holiday_name_encoded'])

# 将均值合并到训练集和测试集
train_df = train_df.merge(holiday_name_product_mean, on=['holiday_name', 'name'], how='left')
test_df = test_df.merge(holiday_name_product_mean, on=['holiday_name', 'name'], how='left')

# 显示计算的均值
holiday_name_product_mean

,holiday_name,name,holiday_name_encoded
0,1848 Revolution Memorial Day (Extra holiday),Apple_10,49.260000
1,1848 Revolution Memorial Day (Extra holiday),Apple_103,6.740000
2,1848 Revolution Memorial Day (Extra holiday),Apple_109,184.080000
3,1848 Revolution Memorial Day (Extra holiday),Apple_133,342.470000
4,1848 Revolution Memorial Day (Extra holiday),Apple_45,134.000000
...,...,...,...
35672,week,Zucchini_5,75.781806
35673,week,Zucchini_6,19.488044
35674,week,Zucchini_7,27.624624
35675,week,Zucchini_8,268.149119


In [12]:
train_df['holiday_name_encoded'].value_counts(), train_df['holiday_name_encoded'].min(), train_df['holiday_name_encoded'].max()

(holiday_name_encoded
 21.789013      5392
 1193.029788    5390
 22.578082      5386
 13.138361      5386
 347.581493     5385
                ... 
 345.970000        1
 35.290000         1
 61.750000         1
 43.760000         1
 14.200000         1
 Name: count, Length: 28966, dtype: int64,
 0.0,
 18034.0025)

In [13]:
# 定义需要处理的列
columns_to_process = ['L1_category_name_en','L2_category_name_en', 'L3_category_name_en', 'L4_category_name_en']

# 对每一列进行处理
for column in columns_to_process:
    # 在 train 数据上按列分组，计算 sales 的均值
    mean_sales = train_df.groupby(column)['sales'].mean().reset_index()
    mean_sales.rename(columns={'sales': f'{column}_mean'}, inplace=True)
    
    # # 对均值进行log平滑处理
    # mean_sales[f'{column}_mean'] = np.log1p(mean_sales[f'{column}_mean'])  # log1p是 log(x+1) 防止log(0)
   
    # 将均值合并到 train 数据中
    train_df = train_df.merge(mean_sales, on=column, how='left')
    
    # 将均值合并到 test 数据中
    test_df = test_df.merge(mean_sales, on=column, how='left')
    if column not in  ['warehouse','L1_category_name_en']:
        # 删除原始列（如果需要）
        train_df.drop(columns=[column], inplace=True)
        test_df.drop(columns=[column], inplace=True)


In [14]:
train_df.columns,test_df.columns,len(train_df.columns),len(test_df.columns)

(Index(['unique_id', 'date', 'warehouse', 'total_orders', 'sales',
        'sell_price_main', 'holiday_name', 'holiday', 'shops_closed',
        'winter_school_holidays', 'school_holidays', 'product_unique_id',
        'name', 'L1_category_name_en', 'year', 'month_sin', 'month_cos',
        'day_sin', 'day_cos', 'is_discount', 'max_discount', 'day',
        'warehouse_day_mean', 'name_day_mean', 'year_encoded',
        'holiday_name_encoded', 'L1_category_name_en_mean',
        'L2_category_name_en_mean', 'L3_category_name_en_mean',
        'L4_category_name_en_mean'],
       dtype='object'),
 Index(['unique_id', 'date', 'warehouse', 'total_orders', 'sell_price_main',
        'holiday_name', 'holiday', 'shops_closed', 'winter_school_holidays',
        'school_holidays', 'product_unique_id', 'name', 'L1_category_name_en',
        'year', 'month_sin', 'month_cos', 'day_sin', 'day_cos', 'is_discount',
        'max_discount', 'day', 'warehouse_day_mean', 'name_day_mean',
        'year_enco

In [15]:
train_unid = train_df['unique_id']
val_unid = test_df['unique_id']
train_date = train_df['date']
val_date = test_df['date']

y_train = train_df['sales']
train_df.drop(['sales'], axis=1, inplace=True)
train_df.drop(columns=['date','unique_id','holiday_name','name','day','product_unique_id'],inplace=True)
test_df.drop(columns=['date','unique_id','holiday_name','name','day','product_unique_id'],inplace=True)

In [16]:
train_df.select_dtypes(include=['float64']).columns.tolist()

['total_orders',
 'sell_price_main',
 'month_sin',
 'month_cos',
 'day_sin',
 'day_cos',
 'max_discount',
 'warehouse_day_mean',
 'name_day_mean',
 'year_encoded',
 'holiday_name_encoded',
 'L1_category_name_en_mean',
 'L2_category_name_en_mean',
 'L3_category_name_en_mean',
 'L4_category_name_en_mean']

In [17]:
from sklearn.preprocessing import StandardScaler

standard_col = ['total_orders',
 'sell_price_main',
]

# 记录训练集和验证集的大小
train_size = len(train_df)
val_size = len(test_df)
# 拼接训练集和验证集
combined_data = pd.concat([train_df, test_df], axis=0)

# 对拼接后的数据进行独热编码
combined_data = pd.get_dummies(combined_data, drop_first=True)

# 拆分回训练集和验证集
train_df = combined_data.iloc[:train_size, :]
test_df = combined_data.iloc[train_size:, :]

# 标准化
scaler = StandardScaler()
train_df.loc[:, standard_col] = scaler.fit_transform(train_df[standard_col])
test_df.loc[:, standard_col] = scaler.transform(test_df[standard_col])
train_df.shape, test_df.shape


((4007367, 32), (47021, 32))

In [18]:
train_df = train_df.copy()
test_df = test_df.copy()
train_df.loc[:, 'unique_id'] = train_unid
test_df.loc[:, 'unique_id'] = val_unid
train_df.loc[:, 'date'] = train_date
test_df.loc[:, 'date'] = val_date
train_df.loc[:, 'sales'] = y_train
train_df.shape, test_df.shape

((4007367, 35), (47021, 34))

In [19]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4007367 entries, 0 to 4007366
Data columns (total 35 columns):
 #   Column                                   Dtype         
---  ------                                   -----         
 0   total_orders                             float64       
 1   sell_price_main                          float64       
 2   holiday                                  bool          
 3   shops_closed                             bool          
 4   winter_school_holidays                   bool          
 5   school_holidays                          bool          
 6   month_sin                                float64       
 7   month_cos                                float64       
 8   day_sin                                  float64       
 9   day_cos                                  float64       
 10  is_discount                              bool          
 11  max_discount                             float64       
 12  warehouse_day_mean               

In [20]:
train_df['sales']

0          16.34
1          12.63
2          34.55
3          34.52
4          35.92
           ...  
4007362    26.56
4007363    27.42
4007364    33.39
4007365    22.88
4007366    32.10
Name: sales, Length: 4007367, dtype: float64

In [21]:
train_df.shape, test_df.shape,train_df.columns

((4007367, 35),
 (47021, 34),
 Index(['total_orders', 'sell_price_main', 'holiday', 'shops_closed',
        'winter_school_holidays', 'school_holidays', 'month_sin', 'month_cos',
        'day_sin', 'day_cos', 'is_discount', 'max_discount',
        'warehouse_day_mean', 'name_day_mean', 'year_encoded',
        'holiday_name_encoded', 'L1_category_name_en_mean',
        'L2_category_name_en_mean', 'L3_category_name_en_mean',
        'L4_category_name_en_mean', 'warehouse_Budapest_1',
        'warehouse_Frankfurt_1', 'warehouse_Munich_1', 'warehouse_Prague_1',
        'warehouse_Prague_2', 'warehouse_Prague_3',
        'L1_category_name_en_Fruit and vegetable',
        'L1_category_name_en_Meat and fish', 'year_year_2021', 'year_year_2022',
        'year_year_2023', 'year_year_2024', 'unique_id', 'date', 'sales'],
       dtype='object'))

In [22]:
train_df

,total_orders,sell_price_main,holiday,shops_closed,winter_school_holidays,school_holidays,month_sin,month_cos,day_sin,day_cos,...,warehouse_Prague_3,L1_category_name_en_Fruit and vegetable,L1_category_name_en_Meat and fish,year_year_2021,year_year_2022,year_year_2023,year_year_2024,unique_id,date,sales
0,0.177560,1.008098,False,False,False,False,1.000000e+00,6.123234e-17,0.897805,-0.440394,...,False,False,False,False,False,False,True,4845,2024-03-10,16.34
1,-0.537475,0.595392,False,False,False,False,5.000000e-01,-8.660254e-01,-0.937752,0.347305,...,False,False,False,True,False,False,False,4845,2021-05-25,12.63
2,0.206194,0.595392,False,False,False,False,-2.449294e-16,1.000000e+00,-0.790776,-0.612106,...,False,False,False,True,False,False,False,4845,2021-12-20,34.55
3,-0.214842,1.008098,False,False,False,False,8.660254e-01,-5.000000e-01,-0.394356,0.918958,...,False,False,False,False,False,True,False,4845,2023-04-29,34.52
4,0.000515,0.661430,False,False,False,False,8.660254e-01,-5.000000e-01,0.201299,0.979530,...,False,False,False,False,True,False,False,4845,2022-04-01,35.92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4007362,1.610051,-0.319586,False,False,False,False,1.224647e-16,-1.000000e+00,-0.897805,-0.440394,...,False,True,False,False,False,True,False,4941,2023-06-21,26.56
4007363,1.017213,-0.319586,False,False,False,False,1.224647e-16,-1.000000e+00,-0.988468,0.151428,...,False,True,False,False,False,True,False,4941,2023-06-24,27.42
4007364,1.785886,-0.319586,False,False,False,False,1.224647e-16,-1.000000e+00,-0.998717,-0.050649,...,False,True,False,False,False,True,False,4941,2023-06-23,33.39
4007365,1.752816,-0.319586,False,False,False,False,1.224647e-16,-1.000000e+00,-0.968077,-0.250653,...,False,True,False,False,False,True,False,4941,2023-06-22,22.88


In [23]:
for df in [train_df, test_df]:    
# 检查缺失值
    missing_values = test_df.isnull().sum()

    # 过滤出有缺失值的列
    missing_values = missing_values[missing_values > 0]

    # 返回结果
    if not missing_values.empty:
        print("存在缺失值的列及缺失个数：")
        print(missing_values)
    else:
        print("DataFrame 中没有缺失值。")

存在缺失值的列及缺失个数：
name_day_mean           158
year_encoded            355
holiday_name_encoded     87
dtype: int64
存在缺失值的列及缺失个数：
name_day_mean           158
year_encoded            355
holiday_name_encoded     87
dtype: int64


In [24]:
train_df.to_csv('train_encode.csv', index=False)
test_df.to_csv('test_encode.csv', index=False)

KeyboardInterrupt: 